# code based on open route services example

## please read: 
https://openrouteservice.org/2187-2/
https://saralgyaan.com/posts/set-passwords-and-secret-keys-in-environment-variables-maclinuxwindows-python-quicktip/
https://openrouteservice.org/dev/#/api-docs/optimization/post

In [1]:
import folium
import pandas as pd
import geopandas as gpd 
from openrouteservice import client
import json
import time
import glob
import rtree

In [2]:
import os

create_isochromes = False

# in .bashrc add export API_KEY="[api key here]"
# so do this:
# nano .bashrc
# export API_KEY="[api key here]"
# ctrl+X, Y

# load jupyter notebook with:
# env API_KEY=$API_KEY jupyter notebook

api_key = os.environ.get('API_KEY') # Provide your personal API key
# you need to make one on the openrouteservices website. 

print(api_key != None) # DO NOT PRINT YOUR API KEY, OR PUSH IT TO GITHUB!!!!

True


### Part 1 - adapted from https://www.linkedin.com/pulse/isochrones-geopandas-paul-whiteside/

In [3]:
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

Join Metro and Regional Train Datasets

In [4]:
train_sf = gpd.read_file("../data/raw/shapefiles/Train_station_location/PTV_train/PTV_METRO_TRAIN_STATION.shp")
trainreg_sf = gpd.read_file("../data/raw/shapefiles/Train_station_location_reg/PTV_train_reg/PTV_REGIONAL_TRAIN_STATION.shp")

In [5]:
train_df = pd.DataFrame(train_sf)
train_df = train_df.drop(['STOP_NAME', 'TICKETZONE', 'ROUTEUSSP', 'geometry'], axis=1)
train_df.shape

(220, 3)

In [6]:
trainreg_df = pd.DataFrame(trainreg_sf)
trainreg_df = trainreg_df.drop(['STOP_NAME', 'geometry'], axis=1)
trainreg_df.shape

(110, 3)

In [7]:
train_jointdf = train_df.append(trainreg_df,ignore_index=True)
train_jointdf.shape

/tmp/ipykernel_4265/2273636930.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_jointdf = train_df.append(trainreg_df,ignore_index=True)


(330, 3)

#### Create Isochromes using Openrouteservice

In [8]:
l_processed = []
def get_isochrones():
    
    for STOP_ID, LATITUDE, LONGITUDE in train_jointdf.values:

        if not STOP_ID in l_processed:
            point = [LATITUDE, LONGITUDE]
            params_iso = {'profile': 'foot-walking',
              'range': [600, 1200, 1800, 2400, 3000],  # 10, 20, 30, 40 and 50 mins
              'segments': 600,
              'attributes': ['total_pop'],  # Get population count for isochrones
              'locations':[point[::-1]]
              }

            try:
                clnt = client.Client(key=api_key)
                r = clnt.isochrones(**params_iso)

                for feature in r['features']:
                    feature['properties']['name'] = STOP_ID

                with open(f'../data/curated/isochrones/{STOP_ID}.json', 'w') as f:
                    f.write(json.dumps(r))
                    l_processed.append(STOP_ID)
            except:
                print(f"Problem processing {STOP_ID}")

            
            time.sleep(2)

In [ ]:
# Call function to find isochrones - not required to call again
if not create_isochromes: 
    get_isochrones()
    create_isochromes = True

#### Join Each Individual Isochrome into a Geopandas Dataframe

In [10]:
l_dfs = []

for file in glob.glob('../data/curated/isochrones/*'):
    with open(file) as json_file:
        data = json.load(json_file)

        gdf = gpd.GeoDataFrame.from_features(data)
        l_dfs.append(gdf)

In [11]:
gdf_isochrones = pd.concat(l_dfs)

In [12]:
gdf_isochrones.shape

(1650, 6)

#### Join Suburb Data and Isochromes for Visualisation

In [13]:
# create centroids for each suburbs
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)

/home/sesshah/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [14]:
gdf_isochrones.set_crs(epsg=7844, inplace=True)

matrix = gpd.sjoin(gdf_isochrones, vic_centroid_sf, how='inner', op='contains')\
    .loc[:,['SA2_NAME21', 'value', 'name']]\
    .set_index(['SA2_NAME21', 'value'])

matrix = matrix.reset_index()
matrix.columns = ['suburb', 'duration_mins', 'train_station_id']
matrix['duration_mins'] = matrix.duration_mins/60
matrix = matrix.groupby(['suburb', 'train_station_id']).duration_mins.min().to_frame().reset_index()

/home/sesshah/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [15]:
matrix

,suburb,train_station_id,duration_mins
0,Abbotsford,19842,50.0
1,Abbotsford,19843,40.0
2,Abbotsford,19854,50.0
3,Abbotsford,19905,50.0
4,Abbotsford,19906,50.0
...,...,...,...
1725,Yarraville,20022,50.0
1726,Yarraville,20023,30.0
1727,Yarraville,20024,30.0
1728,Yarraville,20025,40.0


#### Visualisation of Isochromes and Suburbs

The blue circles represent the location of suburb centeroids within a 20 minute walk of the inputted train stations. The markers represent the location of the train stations. The coloured shapes represent the borders of the respective isochromes of the train stations (i.e. the maximum time taken to walk to the respective train stations of any points inside the blue borders will be 10 minutes, 20 minutes for the green border and so on).

In [16]:

IDs = ["45793", "15351", "19845"]    # more examples can be added but it may make the visualisation more difficult to interpret
minutes = 40

# set up folium map
map = folium.Map(location=[-37.78, 145.29], zoom_start=9)

def style_func(x):

    distance = int(x['properties']['value'])

    if distance == 600:
        return {'fillColor': 'none', 'color': 'blue', 'weight': 5} 
    elif distance == 1200:
        return {'fillColor': 'none', 'color': 'green', 'weight': 5}
    elif distance == 1800:
        return {'fillColor': 'none', 'color': 'yellow', 'weight': 5}
    elif distance == 2400:
        return {'fillColor': 'none', 'color': 'orange', 'weight': 5}
    elif distance == 3000:
        return {'fillColor': 'none', 'color': 'red', 'weight': 5}
    else:
        return {'fillColor': 'none', 'color': 'black', 'weight': 5}


for STOP_ID in IDs:
    with open(f'../data/curated/isochrones/{STOP_ID}.json', 'r') as f:
        geojson = json.loads(f.read())

    point = geojson['features'][0]['properties']['center'][::-1]

    folium.features.GeoJson(geojson, style_function=style_func).add_to(map) 
    folium.map.Marker(point).add_to(map)

    suburbs = matrix.query("train_station_id == @STOP_ID and duration_mins <= @minutes").values[:,0]
    for s in suburbs:
        sub = vic_sf.loc[vic_sf['SA2_NAME21'] == s]
        name = sub['SA2_NAME21']
        pdgeo = sub['geometry'].to_crs('+proj=cea').centroid.to_crs(sub['geometry'].crs)
        folium.Circle([pdgeo.y, pdgeo.x], 250, fill=True).add_child(folium.Popup(name)).add_to(map)

map

#### Join Property Data and Isochromes using Sjoin

In [17]:
property_df = pd.read_csv("../data/raw/property_and_income.csv")
property_df = property_df.rename(columns={"name": "address"})

In [18]:
property_gdf = gpd.GeoDataFrame(property_df, geometry=gpd.points_from_xy(property_df.longitude, property_df.latitude))

In [19]:
gdf_isochrones.set_crs(epsg=7844, inplace=True)
property_gdf.set_crs(epsg=7844, inplace=True)

matrix_property = gpd.sjoin(gdf_isochrones, property_gdf, how='inner', op='contains')

matrix_property = matrix_property.reset_index()
matrix_property = matrix_property.rename(columns={'value': 'duration_mins', 'name': 'train_station_id'})
matrix_property['duration_mins'] = matrix_property.duration_mins/60

/home/sesshah/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [26]:
out_matrix = matrix_property.loc[matrix_property.groupby('address').duration_mins.idxmin()]

In [27]:
out_matrix['duration_mins'].value_counts()

10.0    3833
20.0    3590
30.0    1569
40.0     823
50.0     513
Name: duration_mins, dtype: int64

In [28]:
out_matrix = out_matrix.drop(['Unnamed: 0', 'level_0', 'geometry', 'group_index', 'center', 
                              'total_pop', 'total_pop', 'train_station_id', 'index_right', 'geometry'], axis=1)

In [29]:
# move duration_mins to end of the df
cols = list(out_matrix.columns.values) 
cols.pop(cols.index('duration_mins'))
out_matrix = out_matrix[cols+['duration_mins']]

In [34]:
out_matrix.shape

out_matrix.to_csv('../data/curated/property_isochromes.csv', index=False)